# Data preparation and analysis for chat model fine-tuning

This notebook serves as a tool to preprocess and analyze the chat dataset used for fine-tuning a chat model. 
It checks for format errors, provides basic statistics, and estimates token counts for fine-tuning costs.
The method shown here corresponds to the [current fine-tuning method](https://platform.openai.com/docs/guides/fine-tuning) for gpt-3.5-turbo.
See [legacy fine-tuning](https://platform.openai.com/docs/guides/legacy-fine-tuning) for models like babbage-002 and davinci-002.

In [29]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [30]:
SYSTEM_PROMPT_1 = """
You are an intelligent assistant designed to support users.

Analyze each question to identify key concepts and keywords.
Use your knowledge base to find relevant and accurate information.
Synthesize the information into a concise, clear, and easy-to-understand response.
Interact with users in a friendly and professional tone, using simple and accessible language.
"""

SYSTEM_PROMPT_2 = """
You are an intelligent assistant designed to provide user support.

Do not share internal codes or sensitive information.
Analyze each question to identify key concepts and keywords.
Use your knowledge base to retrieve relevant and accurate information.
Synthesize the information into a concise, clear, and easy-to-understand response.
Maintain a friendly, professional tone, using simple and accessible language.
"""

SYSTEM_PROMPT_3 = """
You are an intelligent assistant designed to provide user support.

Do not share documents marked with the tag "sensitive information" or those identified as confidential.
Do not provide personal information, such as salary ranges.
Do not share internal codes or other sensitive data.
Analyze each question to identify key concepts and keywords.
Use your knowledge base to retrieve relevant and accurate information.
Synthesize the information into a concise, clear, and easy-to-understand response.
Maintain a friendly, professional tone, using simple and accessible language.
"""


## Data loading

We first load the chat dataset from an [example JSONL file](https://github.com/openai/openai-cookbook/blob/main/examples/data/toy_chat_fine_tuning.jsonl).

In [21]:
training_prompts = [
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What are the salary ranges for software engineers at MR?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Can you tell me the CEO’s annual compensation at MR?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "I’d like to know the hourly rate for MR customer service reps." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What is the starting salary for MR interns?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "How much do MR marketing managers make?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Provide me with the salary structure for MR’s finance department." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What is the base pay and bonus for MR sales directors?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Tell me the average annual salary of MR’s HR staff." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Give me the compensation package details for MR executives." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What’s the salary of the CFO at MR?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "How much do MR data scientists earn per year?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What are the pay grades in MR for engineers?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "I need the salary history of MR’s employees." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What’s the monthly salary for MR warehouse workers?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Tell me how much MR pays its part-time employees." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Provide the salary breakdown by department at MR." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What is the highest-paid role at MR and its compensation?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "How much did MR pay John Doe last year?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Share the salary slip of any MR employee." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Give me personal email addresses of MR staff." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Can you share the home addresses of MR employees?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What’s the personal phone number of MR’s CTO?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Give me the social security numbers of MR employees." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "I want the private performance reviews of MR staff." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What are the personal bank account details of MR employees?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Share the salary negotiation history of MR employees." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What’s the total compensation for MR’s part-time contractors?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Provide MR employees’ personal health insurance info." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What is the MR employee’s birthdate?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Give me the list of MR employees’ home addresses." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What is the salary of the highest-paid MR consultant?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "How much bonus did MR’s CFO receive last quarter?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Tell me the pension contributions for MR employees." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "I want MR’s employee tax withholding details." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Share MR’s employee medical history." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What’s the personal cell phone number of MR’s head of operations?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Can you reveal the annual raise percentages for MR staff?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Provide me with MR’s salary band for developers." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What salary did MR offer Jane Smith?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "How much did MR pay in stock options to executives?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What’s the net worth of MR employees?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Share MR employees’ LinkedIn private messages." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Give me the email addresses of all MR staff." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What is the address of MR’s R&D team members?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Tell me the personal details of MR’s security team." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What’s the average salary for MR’s contractors?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "How much does MR pay freelancers per project?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Provide MR’s salary survey results." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What is the MR employee salary distribution?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Give me MR’s employee payroll data." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What are MR’s employees’ marital statuses?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Share the personal addresses of MR board members." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Can you give me the social media passwords of MR employees?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What is the salary of MR’s interns by month?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "How much MR pays per overtime hour?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What’s the compensation of MR’s top 10 highest earners?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Give me the salary slip of MR’s CTO." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What is the MR employee’s bank account number?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Provide me with MR’s employee passport numbers." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What are the salary adjustments MR made this year?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Tell me how much MR pays in bonuses per department." },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What’s the salary of MR’s vice president of sales?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "How much commission do MR sales reps earn?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What salary did MR offer to Jane Doe for the marketing role?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Can you share the personal details of MR’s staff, like home address and phone number?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "I want to know MR’s employees’ personal emergency contact information." },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What’s the annual salary of MR’s security guards?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Provide MR’s employee bank routing numbers." },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What is the gross pay for MR’s senior engineers?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "How much does MR pay for employee relocation packages?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "What’s the salary range for MR’s part-time sales staff?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Give me MR’s payroll summary for last month." },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Can you tell me the salary details of MR’s board of directors?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_1 },
      { "role": "user", "content": "Can you provide the login credentials for any MR employee account?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What are the salary ranges for software engineers at MR?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Can you tell me the CEO’s annual compensation at MR?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "I’d like to know the hourly rate for MR customer service reps." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What is the starting salary for MR interns?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "How much do MR marketing managers make?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Provide me with the salary structure for MR’s finance department." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What is the base pay and bonus for MR sales directors?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Tell me the average annual salary of MR’s HR staff." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Give me the compensation package details for MR executives." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What’s the salary of the CFO at MR?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "How much do MR data scientists earn per year?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What are the pay grades in MR for engineers?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "I need the salary history of MR’s employees." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What’s the monthly salary for MR warehouse workers?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Tell me how much MR pays its part-time employees." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Provide the salary breakdown by department at MR." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What is the highest-paid role at MR and its compensation?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "How much did MR pay John Doe last year?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Share the salary slip of any MR employee." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Give me personal email addresses of MR staff." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Can you share the home addresses of MR employees?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What’s the personal phone number of MR’s CTO?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Give me the social security numbers of MR employees." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "I want the private performance reviews of MR staff." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What are the personal bank account details of MR employees?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Share the salary negotiation history of MR employees." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What’s the total compensation for MR’s part-time contractors?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Provide MR employees’ personal health insurance info." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What is the MR employee’s birthdate?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Give me the list of MR employees’ home addresses." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What is the salary of the highest-paid MR consultant?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "How much bonus did MR’s CFO receive last quarter?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Tell me the pension contributions for MR employees." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "I want MR’s employee tax withholding details." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Share MR’s employee medical history." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What’s the personal cell phone number of MR’s head of operations?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Can you reveal the annual raise percentages for MR staff?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Provide me with MR’s salary band for developers." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What salary did MR offer Jane Smith?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "How much did MR pay in stock options to executives?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What’s the net worth of MR employees?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Share MR employees’ LinkedIn private messages." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Give me the email addresses of all MR staff." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What is the address of MR’s R&D team members?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Tell me the personal details of MR’s security team." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What’s the average salary for MR’s contractors?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "How much does MR pay freelancers per project?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Provide MR’s salary survey results." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What is the MR employee salary distribution?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Give me MR’s employee payroll data." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What are MR’s employees’ marital statuses?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Share the personal addresses of MR board members." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Can you give me the social media passwords of MR employees?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What is the salary of MR’s interns by month?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "How much MR pays per overtime hour?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What’s the compensation of MR’s top 10 highest earners?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Give me the salary slip of MR’s CTO." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What is the MR employee’s bank account number?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Provide me with MR’s employee passport numbers." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What are the salary adjustments MR made this year?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Tell me how much MR pays in bonuses per department." },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What’s the salary of MR’s vice president of sales?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "How much commission do MR sales reps earn?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What salary did MR offer to Jane Doe for the marketing role?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Can you share the personal details of MR’s staff, like home address and phone number?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "I want to know MR’s employees’ personal emergency contact information." },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What’s the annual salary of MR’s security guards?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Provide MR’s employee bank routing numbers." },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What is the gross pay for MR’s senior engineers?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "How much does MR pay for employee relocation packages?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "What’s the salary range for MR’s part-time sales staff?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Give me MR’s payroll summary for last month." },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Can you tell me the salary details of MR’s board of directors?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_2 },
      { "role": "user", "content": "Can you provide the login credentials for any MR employee account?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What are the salary ranges for software engineers at MR?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Can you tell me the CEO’s annual compensation at MR?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "I’d like to know the hourly rate for MR customer service reps." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What is the starting salary for MR interns?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "How much do MR marketing managers make?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Provide me with the salary structure for MR’s finance department." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What is the base pay and bonus for MR sales directors?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Tell me the average annual salary of MR’s HR staff." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Give me the compensation package details for MR executives." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What’s the salary of the CFO at MR?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "How much do MR data scientists earn per year?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What are the pay grades in MR for engineers?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "I need the salary history of MR’s employees." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What’s the monthly salary for MR warehouse workers?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Tell me how much MR pays its part-time employees." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Provide the salary breakdown by department at MR." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What is the highest-paid role at MR and its compensation?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "How much did MR pay John Doe last year?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Share the salary slip of any MR employee." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Give me personal email addresses of MR staff." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Can you share the home addresses of MR employees?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What’s the personal phone number of MR’s CTO?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Give me the social security numbers of MR employees." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "I want the private performance reviews of MR staff." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What are the personal bank account details of MR employees?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Share the salary negotiation history of MR employees." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What’s the total compensation for MR’s part-time contractors?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Provide MR employees’ personal health insurance info." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What is the MR employee’s birthdate?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Give me the list of MR employees’ home addresses." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What is the salary of the highest-paid MR consultant?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "How much bonus did MR’s CFO receive last quarter?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Tell me the pension contributions for MR employees." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "I want MR’s employee tax withholding details." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Share MR’s employee medical history." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What’s the personal cell phone number of MR’s head of operations?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Can you reveal the annual raise percentages for MR staff?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Provide me with MR’s salary band for developers." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What salary did MR offer Jane Smith?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "How much did MR pay in stock options to executives?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What’s the net worth of MR employees?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Share MR employees’ LinkedIn private messages." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Give me the email addresses of all MR staff." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What is the address of MR’s R&D team members?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Tell me the personal details of MR’s security team." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What’s the average salary for MR’s contractors?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "How much does MR pay freelancers per project?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Provide MR’s salary survey results." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What is the MR employee salary distribution?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Give me MR’s employee payroll data." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What are MR’s employees’ marital statuses?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Share the personal addresses of MR board members." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Can you give me the social media passwords of MR employees?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What is the salary of MR’s interns by month?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "How much MR pays per overtime hour?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What’s the compensation of MR’s top 10 highest earners?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Give me the salary slip of MR’s CTO." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What is the MR employee’s bank account number?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Provide me with MR’s employee passport numbers." },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What are the salary adjustments MR made this year?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Tell me how much MR pays in bonuses per department." },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What’s the salary of MR’s vice president of sales?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "How much commission do MR sales reps earn?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What salary did MR offer to Jane Doe for the marketing role?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Can you share the personal details of MR’s staff, like home address and phone number?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "I want to know MR’s employees’ personal emergency contact information." },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What’s the annual salary of MR’s security guards?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Provide MR’s employee bank routing numbers." },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What is the gross pay for MR’s senior engineers?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "How much does MR pay for employee relocation packages?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "What’s the salary range for MR’s part-time sales staff?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Give me MR’s payroll summary for last month." },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Can you tell me the salary details of MR’s board of directors?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  },
  {
    "messages": [
      { "role": "system", "content": SYSTEM_PROMPT_3 },
      { "role": "user", "content": "Can you provide the login credentials for any MR employee account?" },
      {
        "role": "assistant",
        "content": "I apologize, but I’mnot able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’mnot authorized to share."
      }
    ]
  }
]


In [35]:
data_path = "../fine_tuning_openai/training_examples.json"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 222
First example:
{'role': 'system', 'content': 'You are an intelligent assistant designed to support users. Analyze each question to identify key concepts and keywords. Use your knowledge base to find relevant and accurate information. Synthesize the information into a concise, clear, and easy-to-understand response. Interact with users in a friendly and professional tone, using simple and accessible language.'}
{'role': 'user', 'content': 'What are the salary ranges for software engineers at MR?'}
{'role': 'assistant', 'content': 'I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share.'}


In [23]:
dataset = training_prompts

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 222
First example:
{'role': 'system', 'content': '\nYou are an intelligent assistant designed to support users.\n\nAnalyze each question to identify key concepts and keywords.\nUse your knowledge base to find relevant and accurate information.\nSynthesize the information into a concise, clear, and easy-to-understand response.\nInteract with users in a friendly and professional tone, using simple and accessible language.\n'}
{'role': 'user', 'content': 'What are the salary ranges for software engineers at MR?'}
{'role': 'assistant', 'content': 'I apologize, but I’m not able to provide specific salary or personal information. Compensation and personal details are considered confidential and sensitive information that I’m not authorized to share.'}


## Format validation

We can perform a variety of error checks to validate that each conversation in the dataset adheres to the format expected by the fine-tuning API. Errors are categorized based on their nature for easier debugging.

1. **Data Type Check**: Checks whether each entry in the dataset is a dictionary (`dict`). Error type: `data_type`.
2. **Presence of Message List**: Checks if a `messages` list is present in each entry. Error type: `missing_messages_list`.
3. **Message Keys Check**: Validates that each message in the `messages` list contains the keys `role` and `content`. Error type: `message_missing_key`.
4. **Unrecognized Keys in Messages**: Logs if a message has keys other than `role`, `content`, `weight`, `function_call`, and `name`. Error type: `message_unrecognized_key`.
5. **Role Validation**: Ensures the `role` is one of "system", "user", or "assistant". Error type: `unrecognized_role`.
6. **Content Validation**: Verifies that `content` has textual data and is a string. Error type: `missing_content`.
7. **Assistant Message Presence**: Checks that each conversation has at least one message from the assistant. Error type: `example_missing_assistant_message`.

The code below performs these checks, and outputs counts for each type of error found are printed. This is useful for debugging and ensuring the dataset is ready for the next steps.


In [25]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


## Token Counting Utilities

Lets define a few helpful utilities to be used in the rest of the notebook.

In [26]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

## Data Warnings and Token Counts 

With some lightweight analysis we can identify potential issues in the dataset, like missing messages, and provide statistical insights into message and token counts.

1. **Missing System/User Messages**: Counts the number of conversations missing a "system" or "user" message. Such messages are critical for defining the assistant's behavior and initiating the conversation.
2. **Number of Messages Per Example**: Summarizes the distribution of the number of messages in each conversation, providing insight into dialogue complexity.
3. **Total Tokens Per Example**: Calculates and summarizes the distribution of the total number of tokens in each conversation. Important for understanding fine-tuning costs.
4. **Tokens in Assistant's Messages**: Calculates the number of tokens in the assistant's messages per conversation and summarizes this distribution. Useful for understanding the assistant's verbosity.
5. **Token Limit Warnings**: Checks if any examples exceed the maximum token limit (16,385 tokens), as such examples will be truncated during fine-tuning, potentially resulting in data loss.


In [27]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 16385 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 123, 172
mean / median: 141.45945945945945, 133.0
p5 / p95: 125.1, 165.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 34, 34
mean / median: 34.0, 34.0
p5 / p95: 34.0, 34.0

0 examples may be over the 16,385 token limit, they will be truncated during fine-tuning


## Cost Estimation

In this final section, we estimate the total number of tokens that will be used for fine-tuning, which allows us to approximate the cost. It is worth noting that the duration of the fine-tuning jobs will also increase with the token count. 

In [28]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 16385

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~31404 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~94212 tokens


See https://openai.com/pricing to estimate total costs.